In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fer2013/fer2013.csv


In [6]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D,ZeroPadding2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

df=pd.read_csv('/kaggle/input/fer2013/fer2013.csv')

# print(df.info())
# print(df["Usage"].value_counts())

# print(df.head())

#cannot produce



In [7]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_features = 64
num_labels = 7
batch_size = 64
epochs = 200
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [8]:
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# print(f"shape:{X_train.shape}")
##designing the cnn
from keras.activations import elu 
from keras.regularizers import L2
from keras.activations import softmax
from keras.optimizers import Adamax
l2_reg=0

In [9]:
train_y.shape

(28709, 7)

In [10]:
model = Sequential()

#first conv layer
model.add(Conv2D(64,(3,3),input_shape = X_train.shape[1:],padding='same',activation=elu))
model.add(Dropout(0.3))#
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

#second cinv layer
model.add(Conv2D(128,(5,5),padding='same',activation=elu))
model.add(Dropout(0.3))#
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

#third conv layer
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))


#fourth conv layer
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(Dropout(0.3))#
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
#fifth conv layer
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3),padding='same',activation=elu))
model.add(Dropout(0.3))#
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

#first FC layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#second FC layer
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#third FC layer
model.add(Dense(num_labels, activation=softmax))
# model.summary()

from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint = ModelCheckpoint("model.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)
early_stopping = EarlyStopping(monitor="val_loss",
                              min_delta=0,
                              patience=5,
                              verbose=1,
                              restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)
#putting callbacks in callbacks list
callbacks = [checkpoint,early_stopping,reduce_lr]
model.compile(loss="categorical_crossentropy",
             optimizer=Adam(lr=0.001),
             metrics=['accuracy'])
model.summary()

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,validation_data=(X_test, test_y),
          shuffle=True)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [64,7] and labels shape [448]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-10-e6eee096348d>:79) ]] [Op:__inference_train_function_3001]

Function call stack:
train_function
